# The Murnaghan-Nakayama Rule

## Introduction
In representation theory, matrices are a way to model an abstract group that can help us understand the group in detail. More specifically, we have the following definition.

**Definition 1.** A *matrix representation* of a group $G$ is a homomorphism $X: G \to GL_d$ where $GL_d$ is the general linear group of degree $d$ made of invertible $d\times d$ matrices. That is,
1. $X(\epsilon) = I$, where $\epsilon$ is the unity of $G$
2. $X(gh) = X(g)X(h)$

Some representations are built out of smaller ones, while others are indivisible. The latter kind are known as *irreducible representations*.

A representation requires keeping track of a lot of information, but it turns out that one can obtain most of a representation's useful information by looking at one simple statistic: the associated *character* of the representation.


**Definition 2.** Let $X$ be a representation of $G$. Then, the *character* of $X$ for all $g\in G$ is 
$$\chi(g) = tr X(g),$$
where $tr X(g)$ is the trace of the matrix $X(g)$.

If $X$ is an irreducible representation, then $\chi(g)$ is an irreducible character.

**Proposition 1.** Recall that $a, b\in G$ are conjugates if there exists $g\in G$ such that $b = g^{-1}ag$ and the equivalence classes of this equivalence relation are called conjugacy classes. If $K$ is a conjugacy class of $G$, then for all $a, b \in K$, $\chi(b) = \chi(a)$. Thus, we define $\chi_K = \chi(g)$ for any $g\in K$.

*Proof.* By definition, there exists some $g$ such that $b = g^{-1}ag$, so

$$tr X(b) = tr X(g^{-1}ag) = tr X(g^{-1}) X(a) X(g) = tr X(g)^{-1} X(a) X(g) = tr X(a)$$

since trace is invariant under conjugation. $\blacksquare$

**Definition 3.** A *character table* is an array with columns indexed by the conjugacy classes of $G$ and rows indexed by the irreducible characters of $G$. Thus, for a conjugacy class $K$ and an irreducible character $\chi^{(i)}$, entry $(\chi^{(i)}, K)$ is $\chi^{(i)}_K$.

$$
\begin{array}{ c|ccc } 
  & \cdots & K & \cdots \\ 
  \hline
 \vdots & & \vdots & \\ 
 \chi^{(i)} & \cdots & \chi^{(i)}_K & \\
 \vdots & & & 
\end{array}
$$


It is not clear at all that there should be a finite number of irreducible representations $\chi^{(i)}$ should be finite. It turns out, however, that the number of irreducible representations (and characters) of $G$ is equal to the number of conjugacy classes of $G$. That is, the character table is always a square. 

The irreducible representations of $S_n$, and other groups, are not known in general for arbitrary $n$. In this context, the word "modules" is more applicable than "representations". It is known, however, that the irreducible characters of $S_n$ are indexed by a Yound diagram of size $n$, which is a partition of $n$. Thus, the character indices are $\chi^{\lambda}$ for all tableaux $\lambda$ of $n$ elements. 

## The Main Theorem

**Observation 1.** Recall that $\pi_1, \pi_2 \in S_n$, the symmetric group, belong to the same conjugacy class if and only if $\pi_1, \pi_2$ have the same cycle type. Thus, a character can be indexed by its cycle type, which is a partition of $n$.

**Definition 4.** Let $\lambda$ be a tableau and $\mu$ be an integer partition. A *ribbon* $r$ is a skew shape $\lambda / \mu$ that
- does not contain a $2\times 2$ array $~
\left(\begin{array}{ |c|c| } 
  \hline
   & \\
   \hline
   & \\
   \hline
\end{array}\right)
$

- is connected

The *height of a ribbon*, $ht(r)$ is the number of rows in $\lambda / \mu$ minus 1. We will soon see examples of this definition when we implement the function `reduce` below.

**Theorem.** The Murnaghan-Nakayama Rule is an algorithm to compute the irreducible characters of the symmetric group $S_n$. Since the character of a permutation is indexed by a partition and the character is the same for permutations with the same cycle type, we compute $\chi_{\mu}^{\lambda}$ for a cycle type $\mu = (\mu_1, \ldots, \mu_{\ell})$ and a tableau $\lambda$ as follows

$$\chi_{()}^{()} = 1$$

$$\chi_{\mu}^{\lambda} = \sum_{r\text{ ribbon of size }\mu_1} (-1)^{ht(r)}\chi_{(\mu_2, \ldots, \mu_{\ell})}^{\lambda / r}$$

where $\lambda / r$ is the tableau $\lambda$ with the removed ribbon.


## Implementation

In [1]:
# Some useful, pretty print functions

# Params: shape - non-increasing integer list specifying a tableau's shape
# Prints a tableau of given shape
def print_tableau(shape):
    print(" ___" * shape[0])
    for l in shape:
        print("|" + "   |" * l)
        print("|" + "___|" * l)

# Params: original - non-increasing integer list specifying the shape of original tableau
#         reduced - non-increasing integer list specifying the shape of "reduced" tableau (with removed ribbon)
# Prints reduced tableau
def print_red_tableau(original, reduced, tab=""):
    print(tab + " ___" * original[0])
    for r in range(len(original)):
        l = original[r]
        cells = tab + "|"
        for i in range(l):
            cells += " " + (" " if (r < len(reduced) and i < reduced[r]) else "X") + " |"
        print(cells)
        print(tab + "|" + "___|" * l)

First, we need a way to compute the ribbons of a tableau. More specifically, we need to compute $ht(r)$ and $\lambda / r$ for a given tableau $\lambda$ and a ribbon $r$ of size $s$. For this, we define the function `reduce` below, which takes a shape $\lambda$ and a size $s$ and returns a list of pairs $(\lambda / r, ht(r))$ for all ribbons $r$ of $\lambda$ of size $s$.

In [2]:
# Params: shape - 𝜆, a non-increasing integer list
#         s - 𝑠, size of a ribbon
# Return: A list of tuples (𝜆/𝑟, ht(𝑟)) for all ribbons 𝑟 of 𝜆 of size 𝑠
def reduce(shape, s):
    th_list = [] # list of (𝜆∖𝑟, ℎ(𝑟)) pairs
    
    border_size = [] # list containing number of border cells in every row
    for r in range(len(shape)):
        border_size.append(shape[r] if r == len(shape) - 1
                           else shape[r] - shape[r + 1] + 1)
    
    for r_start_idx in range(len(shape)):
        new_shape = shape.copy()
        
        removed = 0 # number of removed border cells
        r_idx = r_start_idx # index of row from which to remove cells
        while removed < s and r_idx < len(shape):
            rrem = min(border_size[r_idx], s - removed) # number of cells to remove from row at rem_idx
            new_shape[r_idx] -= rrem # remove cells from shape
            
            removed += rrem
            r_idx += 1
        
        # check that shape is valid and that exactly s cells have been removed
        if removed != s or (r_idx < len(shape) and new_shape[r_idx - 1] < new_shape[r_idx]):
            continue
        else:
            # remove 0s from new shape
            new_shape = list(filter(lambda x: x != 0, new_shape))
            th_list.append((new_shape, r_idx - r_start_idx - 1))
    
    return th_list

# Example
def show_reduced_tableaux(shape, s):
    print("An example with 𝜆 = " + str(tuple(shape)) + " and 𝑠 = " + str(s))
    print("This example shows all 𝜆/𝑟 for ribbons 𝑟 size 𝑠 (marked with X)" )
    print_tableau(shape)
    th_list = reduce(shape, s)
    
    for t, h in th_list:
        print_red_tableau(shape, t)
        print("height: ", h)

show_reduced_tableaux([5, 4, 4, 3], 4)

An example with 𝜆 = (5, 4, 4, 3) and 𝑠 = 4
This example shows all 𝜆/𝑟 for ribbons 𝑟 size 𝑠 (marked with X)
 ___ ___ ___ ___ ___
|   |   |   |   |   |
|___|___|___|___|___|
|   |   |   |   |
|___|___|___|___|
|   |   |   |   |
|___|___|___|___|
|   |   |   |
|___|___|___|
 ___ ___ ___ ___ ___
|   |   |   | X | X |
|___|___|___|___|___|
|   |   |   | X |
|___|___|___|___|
|   |   |   | X |
|___|___|___|___|
|   |   |   |
|___|___|___|
height:  2
 ___ ___ ___ ___ ___
|   |   |   |   |   |
|___|___|___|___|___|
|   |   |   | X |
|___|___|___|___|
|   |   | X | X |
|___|___|___|___|
|   |   | X |
|___|___|___|
height:  2
 ___ ___ ___ ___ ___
|   |   |   |   |   |
|___|___|___|___|___|
|   |   |   |   |
|___|___|___|___|
|   |   | X | X |
|___|___|___|___|
|   | X | X |
|___|___|___|
height:  1


In the example above, the ribbons of size $4$ of the Young diagram $\lambda = (5, 4, 4, 3)$ are marked with a path of `X`'s.


Now, we implement the algorithm in the recursive function `character` below, which takes as input two partitions $\lambda$ and $\mu$ and returns $\chi_{\mu}^{\lambda}$.

Note that, since this is a recursive algorithm, we might be computing the same character values for a given $\lambda$ and $\mu$  multiple times, which might decrease the computational efficiency of the implementation. Therefore, if we have already computed $\chi_{\mu}^{\lambda}$, then we can store, or *memoize* this value in the dictionary `memo` below so that the same value is not computed again.

In [3]:
from IPython.display import display, Math

memo = {} # dictionary used to memoize already computed character values, 
          # so as to avoid multiple computation of the same characters

# Params: lda - 𝜆, non-increasing integer list
#         mu - 𝜇, non-increasing integer list
#         idx - (defaults to 0) index at which rho starts
#         print_flag - (defaults to False) whether to print the tableaux at every step
# Returns 𝜒𝜆_𝜌
def character(lda, mu, idx=0, print_flag=False):
    key = (tuple(lda), tuple(mu)) # to index into memo
    
    if key in memo: # return from memo if already computed
        if print_flag:
            print("\t\t"*(idx + 1) + "returns ", memo[key], "from memo")
        return memo[key]
    
    if idx == 0 and print_flag:
        print_tableau(lda)
    if len(lda) == 0 and idx == len(mu):
        if print_flag:
            print("\t\t"*(idx + 1) + "⊥")
            print("\t\t"*(idx + 1) + "returns 1")
        return 1
    
    th_list = reduce(lda, mu[idx])
    char = 0
    
    for t, h in th_list:
        if print_flag:
            print_red_tableau(lda, t, tab="\t\t"*(idx + 1))
        
        ind = -1 if h % 2 == 1 else 1
        char_t = character(t, mu, idx + 1, print_flag)
        char += ind * char_t
    
        if print_flag:
            print("\t\t"*(idx + 1) + "returns ", char_t)
    
    memo[key] = char
    return char

# Example
print("")
display(Math(r'Example: \chi_{(3, 3, 1, 1)}^{(5, 2, 1)}'))
print("\nCharacter value: ", character([5, 2, 1], [3, 3, 2], print_flag=True))

<IPython.core.display.Math object>

 ___ ___ ___ ___ ___
|   |   |   |   |   |
|___|___|___|___|___|
|   |   |
|___|___|
|   |
|___|
		 ___ ___ ___ ___ ___
		|   |   | X | X | X |
		|___|___|___|___|___|
		|   |   |
		|___|___|
		|   |
		|___|
				 ___ ___
				|   |   |
				|___|___|
				| X | X |
				|___|___|
				| X |
				|___|
						 ___ ___
						| X | X |
						|___|___|
								⊥
								returns 1
						returns  1
				returns  1
		returns  -1
		 ___ ___ ___ ___ ___
		|   |   |   |   |   |
		|___|___|___|___|___|
		| X | X |
		|___|___|
		| X |
		|___|
				 ___ ___ ___ ___ ___
				|   |   | X | X | X |
				|___|___|___|___|___|
						returns  1 from memo
				returns  1
		returns  1

Character value:  -2


In the example of a character computation above, the indentation shows the depth of the the Yound diagram in the recursive tree. One thing to note is that in the 3rd level we have 2 Young diagrams with shape $(2)$. By memoizing the character values, we can compute the value for the first diagram and simply retrieve it from the dictionary for the second one.

Now, we implement a method to compute the character table of $S_n$ for a given $n$.

In [4]:
# Helper method for partitions below
def partitions_recursive(n, max_size):
    if n == 0:
        return [[]]
    
    max_size = min(max_size, n)
    p = []
    for i in range(max_size, 0, -1):
        p_pr = partitions_recursive(n - i, i)
        for part in p_pr:
            part.append(i)
            p.append(part)
    
    return p

# Params: n - int
# Returns: list of integer partitions of n (as non-increasing integer lists)
def partitions(n):
    p = partitions_recursive(n, n)
    for part in p:
        part.reverse()
    return p

In [5]:
from pandas import DataFrame

# Params: n - int
# Return: character table for 𝑆𝑛, as a pandas DataFrame indexed by partitions of 𝑛
def character_table(n):
    p = partitions(n)
    n_parts = len(p)
    mat = {}
    
    for j in range(n_parts):
        mat_row = []
        for i in range(n_parts):
            mat_row.append(character(list(p[i]), list(p[j])))
        mat[str(tuple(p[j]))] = mat_row
    
    return DataFrame(mat, index=[tuple(part) for part in p])

### Examples

In [6]:
print("Character table of 𝑆_3")
character_table(3)

Character table of 𝑆_3


,"(3,)","(2, 1)","(1, 1, 1)"
"(3,)",1,1,1
"(2, 1)",-1,0,2
"(1, 1, 1)",1,-1,1


In [8]:
print("Character table of 𝑆_10")
character_table(10)

Character table of 𝑆_10


,"(10,)","(9, 1)","(8, 2)","(8, 1, 1)","(7, 3)","(7, 2, 1)","(7, 1, 1, 1)","(6, 4)","(6, 3, 1)","(6, 2, 2)",...,"(3, 2, 2, 2, 1)","(3, 2, 2, 1, 1, 1)","(3, 2, 1, 1, 1, 1, 1)","(3, 1, 1, 1, 1, 1, 1, 1)","(2, 2, 2, 2, 2)","(2, 2, 2, 2, 1, 1)","(2, 2, 2, 1, 1, 1, 1)","(2, 2, 1, 1, 1, 1, 1, 1)","(2, 1, 1, 1, 1, 1, 1, 1, 1)","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)"
"(10,)",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"(9, 1)",-1,0,-1,1,-1,0,2,-1,0,-1,...,0,2,4,6,-1,1,3,5,7,9
"(8, 2)",0,-1,1,-1,0,0,0,0,-1,2,...,2,2,6,14,5,3,5,11,21,35
"(8, 1, 1)",1,0,0,0,1,-1,1,1,0,-1,...,-3,-1,5,15,-4,-4,0,8,20,36
"(7, 3)",0,0,-1,-1,1,0,-2,0,1,-2,...,1,3,5,15,-5,3,7,15,35,75
"(7, 2, 1)",0,1,0,0,-1,1,-1,0,0,0,...,0,-2,4,34,0,0,0,16,64,160
"(7, 1, 1, 1)",-1,0,0,0,0,0,0,-1,1,1,...,1,-3,1,21,4,-4,-8,0,28,84
"(6, 4)",0,0,0,0,-1,-1,-1,1,0,1,...,0,2,4,6,10,2,6,14,34,90
"(6, 3, 1)",0,0,1,1,0,0,0,-1,0,1,...,-3,1,1,21,-5,-5,3,19,91,315
"(6, 2, 2)",0,0,-1,1,1,-1,1,0,0,0,...,3,-1,-5,15,15,9,3,5,55,225


I ran the program for different values of $n$ and kept track of how much time it took for the program to run with Python's `time` module:

```python
import time

start_time = time.time()
character_table(n)
print("Total time:", time.time() - start_time)
```

The program runs quickly (in less than a second) for $n \leq 20$ and takes under a minute for slightly larger values of $n$.

## Conclusion

Much of the information contained in a representation is encoded by the representation's character. For example, two representations are equivalent if and only if they have the same character. This is why computing the characters of a group is of interest in the field of representation theory.

In general, computing character tables is not an easy task. For $S_n$, however, the Murnaghan-Nakayama Rule provides a way of easily and somewhat efficiently computing these tables.

## References

- Sagan, B. E. (1991). *The Symmetric Group*. Springer.
- Math 68 (2019) [lecture notes](https://math.dartmouth.edu/~m68f19/general.php)
